# Modelado tabular con Autgluon

In [1]:
!sudo apt-get update
!sudo apt-get install gcsfuse

Hit:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:2 https://deb.debian.org/debian bullseye InRelease              
Hit:3 https://download.docker.com/linux/debian bullseye InRelease   
Get:4 https://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Hit:5 https://deb.debian.org/debian bullseye-updates InRelease
Hit:6 https://deb.debian.org/debian bullseye-backports InRelease
Hit:7 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:8 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Hit:10 https://packages.cloud.google.com/apt google-fast-socket InRelease
Fetched 27.2 kB in 1s (32.7 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcsfuse is already the newest version (3.1.0).
0 upgraded, 0 newly installed, 0 to remove and 6 not u

In [2]:
#!pip install autogluon.timeseries

# Carga librerías

In [2]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor
from autogluon.tabular import TabularPredictor
import numpy as np

In [3]:
!mkdir -p /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top150_v3

In [3]:
!fusermount -u /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top150_v3

fusermount: entry for /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top150_v3 not found in /etc/mtab


In [4]:
!gcsfuse forecasting_customer_product /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top150_v3

{"timestamp":{"seconds":1752957183,"nanos":833161408},"severity":"INFO","message":"Start gcsfuse/3.1.0 (Go version go1.24.0) for app \"\" using mount point: /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top150_v3\n"}
{"timestamp":{"seconds":1752957183,"nanos":833208216},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"DisableAutoconfig":false,"EnableAtomicRenameObject":true,"EnableGoogleLibAuth":false,"EnableHns":true,"EnableNewReader":false,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":200,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"ExperimentalExcludeRegex":"","ExperimentalParallelDownloadsDefaultOn":true,"MaxParallelDownloads":96,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"ExperimentalEnableDentryCache":false,"Exper

In [5]:
# ------------------------
# 1) Cargar parquet con FE
# ------------------------

parquet_path = "panel_cliente_producto_fe.parquet"
df_modelo = pd.read_parquet(parquet_path)

# ------------------------
# 2) Cargar Prophet features
# ------------------------
csv_path = "prophet_features_customer_product.csv"
df_prophet = pd.read_csv(csv_path)
print(f"✅ CSV Prophet cargado. Shape: {df_prophet.shape}")

# ------------------------
# 3) Asegurar consistencia de tipos
# ------------------------
df_modelo['fecha'] = pd.to_datetime(df_modelo['fecha'])
df_prophet['fecha'] = pd.to_datetime(df_prophet['fecha'])

# ------------------------
# 4) Realizar join por 'product_id' y 'fecha'
# ------------------------
df_modelo_final = df_modelo.merge(
    df_prophet,
    on=['customer_id','product_id', 'fecha'],
    how='left'
)
print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# ------------------------
# 5) Convertir columnas float64 a float32 para ahorrar memoria
# ------------------------
float64_cols = df_modelo_final.select_dtypes(include=['float64']).columns.tolist()

df_modelo_final[float64_cols] = df_modelo_final[float64_cols].astype('float32')

print(f"✅ Conversión de float64 a float32 completada para columnas: {float64_cols}")
print(df_modelo_final.info())

print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# Verifica el resultado
df_modelo_final.head()


print(f"✅ Parquet cargado. Shape: {df_modelo_final.shape}")

✅ CSV Prophet cargado. Shape: (7249573, 9)
✅ Merge completado. Shape final: (12138186, 200)
✅ Conversión de float64 a float32 completada para columnas: ['tn_x', 'inflacion', 'cambio_dolar', 'stock_final', 'clase', 'tn_1', 'diff_tn_1', 'tn_2', 'diff_tn_2', 'tn_3', 'diff_tn_3', 'tn_4', 'diff_tn_4', 'tn_5', 'diff_tn_5', 'tn_6', 'diff_tn_6', 'tn_7', 'diff_tn_7', 'tn_8', 'diff_tn_8', 'tn_9', 'diff_tn_9', 'tn_10', 'diff_tn_10', 'tn_11', 'diff_tn_11', 'tn_12', 'diff_tn_12', 'tn_13', 'diff_tn_13', 'tn_14', 'diff_tn_14', 'tn_15', 'diff_tn_15', 'tn_16', 'diff_tn_16', 'tn_17', 'diff_tn_17', 'tn_18', 'diff_tn_18', 'tn_19', 'diff_tn_19', 'tn_20', 'diff_tn_20', 'tn_21', 'diff_tn_21', 'tn_22', 'diff_tn_22', 'tn_23', 'diff_tn_23', 'tn_24', 'diff_tn_24', 'tn_25', 'diff_tn_25', 'tn_26', 'diff_tn_26', 'tn_27', 'diff_tn_27', 'tn_28', 'diff_tn_28', 'tn_29', 'diff_tn_29', 'tn_30', 'diff_tn_30', 'tn_31', 'diff_tn_31', 'tn_32', 'diff_tn_32', 'tn_33', 'diff_tn_33', 'tn_34', 'diff_tn_34', 'tn_35', 'diff_tn_35',

In [6]:
# =============================================
# 📦 BLOQUE 1 — Filtrar TOP 70 clientes y resto
# =============================================

# 2) Calcular compra promedio por cliente
cliente_avg = (
    df_modelo_final.groupby('customer_id')['tn_x']
    .mean()
    .reset_index()
    .rename(columns={'tn_x': 'avg_tn'})
    .sort_values('avg_tn', ascending=False)
)

# 3) Identificar TOP 20 clientes
top_40_customers = cliente_avg.head(150)['customer_id'].tolist()
print(f"TOP 40 clientes:\n{top_40_customers}")

# 4) Crear datasets
df_top40 = df_modelo_final[df_modelo_final['customer_id'].isin(top_40_customers)].copy()
df_otros = df_modelo_final[~df_modelo_final['customer_id'].isin(top_40_customers)].copy()

print(f"TOP40 shape: {df_top40.shape} | Otros shape: {df_otros.shape}")


TOP 40 clientes:
[10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10015, 10017, 10018, 10019, 10020, 10023, 10022, 10021, 10024, 10025, 10039, 10026, 10028, 10030, 10033, 10027, 10034, 10032, 10031, 10038, 10035, 10037, 10036, 10042, 10041, 10082, 10029, 10045, 10044, 10043, 10040, 10047, 10046, 10074, 10051, 10048, 10050, 10057, 10049, 10053, 10055, 10054, 10127, 10052, 10060, 10056, 10064, 10061, 10062, 10059, 10063, 10058, 10066, 10065, 10067, 10069, 10072, 10071, 10068, 10076, 10073, 10070, 10075, 10081, 10080, 10084, 10086, 10088, 10077, 10078, 10090, 10089, 10079, 10091, 10083, 10085, 10097, 10096, 10087, 10094, 10092, 10093, 10101, 10099, 10098, 10095, 10102, 10104, 10105, 10109, 10113, 10103, 10107, 10108, 10110, 10112, 10106, 10136, 10111, 10115, 10118, 10117, 10121, 10100, 10114, 10116, 10130, 10119, 10122, 10125, 10120, 10124, 10123, 10129, 10126, 10133, 10131, 10139, 10138, 10143, 10142, 10137, 10135, 10132, 10134, 10

In [7]:
# ====================================================
# 🚀 BLOQUE 2 — Modelo TABULAR para TOP 40 clientes
# ====================================================

# ⚙️ Separar Train/Test
df_top40['fecha'] = pd.to_datetime(df_top40['fecha'])
train_top40 = df_top40[(df_top40['fecha'] <= '2019-10-01') & df_top40['clase'].notnull()].copy()
test_top40 = df_top40[df_top40['fecha'] == '2019-12-01'].copy()

# Escalar magnitud de toneladas vendidas
train_top40['sample_weight'] = train_top40['tn_x']

features_top40 = [col for col in df_top40.columns if col not in ['periodo', 'clase', 'tn_y','seasonal']]

# ⚙️ Entrenar predictor
predictor_top40 = TabularPredictor(label='clase', problem_type='regression', eval_metric='mae',
    path='gcs_model_dir_fullpower_hibrido_top150_v3')
predictor_top40.fit(
    train_data=train_top40[features_top40 + ['clase']],
    presets='best_quality',
    time_limit=10800,
    ag_args_fit={'sample_weight': 'sample_weight'}
)

# ⚙️ Predicción y agregado por producto
test_top40['tn_pred'] = predictor_top40.predict(test_top40[features_top40])
df_top40_pred = (
    test_top40.groupby('product_id')['tn_pred']
    .sum()
    .reset_index()
    .rename(columns={'tn_pred': 'tn'})
)
print(df_top40_pred.head())

# ⚙️ Guardar CSV parcial
df_top40_pred.to_csv('forecast_top150_202002_v3.csv', index=False)
print("✅ Forecast TOP100 guardado.")


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          48
Memory Avail:       310.53 GB / 377.89 GB (82.2%)
Disk Space Avail:   1048576.00 GB / 1048576.00 GB (100.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=19655) [1000]	valid_set's l1: 0.244261
(_ray_fit pid=19658) [1000]	valid_set's l1: 0.241968 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=19652) [1000]	valid_set's l1: 0.243346 [repeated 3x across cluster]
(_ray_fit pid=19653) [2000]	valid_set's l1: 0.239813
(_ray_fit pid=19655) [2000]	valid_set's l1: 0.241575
(_ray_fit pid=19658) [2000]	valid_set's l1: 0.239635 [repeated 3x across cluster]
(_ray_fit pid=19654) [2000]	valid_set's l1: 0.245749
(_ray_fit pid=19656) [2000]	valid_set's l1: 0.240813
(_ray_fit pid=19653) [3000]	valid_set's l1: 0.238454 [repeated 2x across cluster]
(_ray_fit pid=19659) [3000]	valid_set's l1: 0.236942 [repeated 2x across cluster]
(_ray_fit pid=19658) [3000]	valid_set's l1: 0.238537 [repeated 2x across cluster]
(_ray_fit pid=19654) [3000]	va

(_ray_fit pid=19656) 	Ran out of time, early stopping on iteration 7429. Best iteration is:
(_ray_fit pid=19656) 	[7390]	valid_set's l1: 0.236889
(_dystack pid=15633) 	-0.237	 = Validation score   (-mean_absolute_error)
(_dystack pid=15633) 	1378.17s	 = Training   runtime
(_dystack pid=15633) 	819.21s	 = Validation runtime
(_ray_fit pid=19658) 	Ran out of time, early stopping on iteration 7592. Best iteration is: [repeated 7x across cluster]
(_ray_fit pid=19658) 	[7574]	valid_set's l1: 0.235997 [repeated 7x across cluster]
(_dystack pid=15633) Fitting model: LightGBM_BAG_L1 ... Training model for up to 229.40s of the 1115.29s of remaining time.
(_dystack pid=15633) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=6, gpus=0, memory=4.09%)
(_ray_fit pid=30676) 	Ran out of time, early stopping on iteration 886. Best iteration is:
(_ray_fit pid=30676) 	[878]	valid_set's l1: 0.242282
(_dystack pid=15633) 	-0.2429	 = Validation score

(_ray_fit pid=32238) [1000]	valid_set's l1: 0.23777 [repeated 2x across cluster]
(_ray_fit pid=32239) [1000]	valid_set's l1: 0.233799 [repeated 4x across cluster]
(_ray_fit pid=32240) [1000]	valid_set's l1: 0.241759 [repeated 3x across cluster]
(_ray_fit pid=32238) [2000]	valid_set's l1: 0.236605
(_ray_fit pid=32242) [2000]	valid_set's l1: 0.241639
(_ray_fit pid=32245) [2000]	valid_set's l1: 0.23335 [repeated 2x across cluster]
(_ray_fit pid=32239) [2000]	valid_set's l1: 0.232524
(_ray_fit pid=32241) [2000]	valid_set's l1: 0.237499
(_ray_fit pid=32238) [3000]	valid_set's l1: 0.236186 [repeated 3x across cluster]
(_ray_fit pid=32241) [3000]	valid_set's l1: 0.237096 [repeated 3x across cluster]
(_ray_fit pid=32240) [3000]	valid_set's l1: 0.239741 [repeated 2x across cluster]


(_ray_fit pid=32240) 	Ran out of time, early stopping on iteration 3376. Best iteration is:
(_ray_fit pid=32240) 	[3263]	valid_set's l1: 0.23959
(_dystack pid=15633) 	-0.2372	 = Validation score   (-mean_absolute_error)
(_dystack pid=15633) 	641.65s	 = Training   runtime
(_dystack pid=15633) 	106.15s	 = Validation runtime
(_ray_fit pid=32241) 	Ran out of time, early stopping on iteration 3454. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=32241) 	[3022]	valid_set's l1: 0.237073 [repeated 4x across cluster]
(_dystack pid=15633) Fitting model: LightGBM_BAG_L2 ... Training model for up to 136.26s of the 135.31s of remaining time.
(_dystack pid=15633) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=6, gpus=0, memory=4.17%)
(_ray_fit pid=41460) 	Ran out of time, early stopping on iteration 411. Best iteration is:
(_ray_fit pid=41460) 	[404]	valid_set's l1: 0.236201
(_dystack pid=15633) 	-0.2369	 = Validation score  

   product_id           tn
0       20001  1301.626465
1       20002  1091.926025
2       20003   730.439636
3       20004   449.942505
4       20005   406.428040
✅ Forecast TOP100 guardado.


In [7]:
# ====================================================
# 🚀 BLOQUE 3 — Modelo TABULAR para resto clientes
# ====================================================

# ⚙️ Separar Train/Test
df_otros['fecha'] = pd.to_datetime(df_otros['fecha'])
train_resto = df_otros[(df_otros['fecha'] <= '2019-10-01') & df_otros['clase'].notnull()].copy()
test_resto = df_otros[df_otros['fecha'] == '2019-12-01'].copy()

# Escalar magnitud de toneladas vendidas
train_resto['sample_weight'] = train_resto['tn_x']

features_resto = [col for col in df_otros.columns if col not in ['periodo', 'clase', 'tn_y','seasonal']]

# ⚙️ Entrenar predictor
predictor_resto = TabularPredictor(label='clase', problem_type='regression', eval_metric='mae',
    path='gcs_model_dir_fullpower_hibrido_top150_v3')
predictor_resto.fit(
    train_data=train_resto[features_resto + ['clase']],
    presets='best_quality',
    time_limit=10800,
    ag_args_fit={'sample_weight': 'sample_weight'}
)

# ⚙️ Predicción y agregado por producto
test_resto['tn_pred'] = predictor_resto.predict(test_resto[features_resto])
df_resto_pred = (
    test_resto.groupby('product_id')['tn_pred']
    .sum()
    .reset_index()
    .rename(columns={'tn_pred': 'tn'})
)
print(df_resto_pred.head())

# ⚙️ Guardar CSV parcial
df_resto_pred.to_csv('forecast_resto_top150_202002_v3.csv', index=False)
print("✅ Forecast TOP100 guardado.")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          48
Memory Avail:       303.24 GB / 377.89 GB (80.2%)
Disk Space Avail:   1048576.00 GB / 1048576.00 GB (100.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=19117) [1000]	valid_set's l1: 0.0126862
(_ray_fit pid=19119) [1000]	valid_set's l1: 0.0127624 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=19118) [1000]	valid_set's l1: 0.0127661
(_ray_fit pid=19117) [2000]	valid_set's l1: 0.0125771
(_ray_fit pid=19119) [2000]	valid_set's l1: 0.0126589 [repeated 2x across cluster]
(_ray_fit pid=19118) [2000]	valid_set's l1: 0.0126649
(_ray_fit pid=19117) [3000]	valid_set's l1: 0.0125199
(_ray_fit pid=19120) [3000]	valid_set's l1: 0.0124774
(_ray_fit pid=19119) [3000]	valid_set's l1: 0.0126119
(_ray_fit pid=19118) [3000]	valid_set's l1: 0.012599


(_ray_fit pid=19117) 	Ran out of time, early stopping on iteration 3494. Best iteration is:
(_ray_fit pid=19117) 	[3494]	valid_set's l1: 0.0124968


(_ray_fit pid=28308) [1000]	valid_set's l1: 0.0126449
(_ray_fit pid=28308) [2000]	valid_set's l1: 0.0125595 [repeated 4x across cluster]
(_ray_fit pid=28308) [3000]	valid_set's l1: 0.0124964 [repeated 4x across cluster]
(_ray_fit pid=28461) [3000]	valid_set's l1: 0.0124802 [repeated 3x across cluster]


(_ray_fit pid=28308) 	Ran out of time, early stopping on iteration 3329. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=28308) 	[3321]	valid_set's l1: 0.0124889 [repeated 4x across cluster]
(_dystack pid=14846) 	-0.0126	 = Validation score   (-mean_absolute_error)
(_dystack pid=14846) 	1340.51s	 = Training   runtime
(_dystack pid=14846) 	182.63s	 = Validation runtime
(_ray_fit pid=28453) 	Ran out of time, early stopping on iteration 3320. Best iteration is: [repeated 3x across cluster]
(_ray_fit pid=28453) 	[3311]	valid_set's l1: 0.0125775 [repeated 3x across cluster]
(_dystack pid=14846) Fitting model: LightGBM_BAG_L1 ... Training model for up to 263.53s of the 1132.23s of remaining time.
(_dystack pid=14846) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 10.46% memory usage per fold, 41.83%/80.00% total).
(_dystack pid=14846) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 worker

(_ray_fit pid=32014) [1000]	valid_set's l1: 0.0126652
(_ray_fit pid=32016) [1000]	valid_set's l1: 0.0123921
(_ray_fit pid=32013) [1000]	valid_set's l1: 0.0125863 [repeated 2x across cluster]


(_ray_fit pid=32015) 	Ran out of time, early stopping on iteration 1401. Best iteration is:
(_ray_fit pid=32015) 	[1399]	valid_set's l1: 0.0124151


(_ray_fit pid=33104) [1000]	valid_set's l1: 0.0126837
(_ray_fit pid=33032) [1000]	valid_set's l1: 0.0124809


(_ray_fit pid=33018) 	Ran out of time, early stopping on iteration 1350. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=33018) 	[1350]	valid_set's l1: 0.0126457 [repeated 4x across cluster]
(_dystack pid=14846) 	-0.0125	 = Validation score   (-mean_absolute_error)
(_dystack pid=14846) 	622.57s	 = Training   runtime
(_dystack pid=14846) 	77.37s	 = Validation runtime
(_ray_fit pid=33104) 	Ran out of time, early stopping on iteration 1402. Best iteration is: [repeated 3x across cluster]
(_ray_fit pid=33104) 	[1397]	valid_set's l1: 0.012656 [repeated 3x across cluster]
(_dystack pid=14846) Fitting model: LightGBM_BAG_L2 ... Training model for up to 114.72s of the 112.39s of remaining time.
(_dystack pid=14846) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 10.76% memory usage per fold, 43.03%/80.00% total).
(_dystack pid=14846) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, p

   product_id         tn
0       20001  36.689022
1       20002  30.132318
2       20003  37.661961
3       20004  53.916893
4       20005  69.847389
✅ Forecast TOP100 guardado.


In [8]:
# ================================================
# 🗃️ BLOQUE 4 — Merge forecasts y salida final
# ================================================

df_top40_pred = pd.read_csv('forecast_top150_202002_v3.csv')
df_resto_pred = pd.read_csv('forecast_resto_top150_202002_v3.csv')

# ⚙️ Unir y sumar por producto
df_final = (
    pd.concat([df_top40_pred, df_resto_pred], axis=0)
    .groupby('product_id', as_index=False)
    .agg({'tn': 'sum'})
)

print(df_final.head())

# ⚙️ Guardar archivo final
df_final.to_csv('forecast_total_top_150_202002_v3.csv', index=False)
print("✅ Forecast combinado guardado: forecast_total_202002.csv")
print(f"Productos únicos: {df_final['product_id'].nunique()} | TN totales: {df_final['tn'].sum():,.2f}")


   product_id           tn
0       20001  1338.315522
1       20002  1122.058318
2       20003   768.101600
3       20004   503.859393
4       20005   476.275430
✅ Forecast combinado guardado: forecast_total_202002.csv
Productos únicos: 780 | TN totales: 26,793.76
